In [ ]:
import os
import polars as pl
from src import IOFunctions
from src import AnalysisFunctions
from src import RASPRoutines
import matplotlib.pyplot as plt
import numpy as np
from src import PlottingFunctions
import numpy as np

RASP = RASPRoutines.RASP_Routines()
IO = IOFunctions.IO_Functions()
A_F = AnalysisFunctions.Analysis_Functions()

In [ ]:
def crop_coord_gen(image_size, crop_size, location="center"):
    if location == "center":
        x_min = int(image_size[0] / 2) - int(crop_size / 2)
        x_max = int(image_size[0] / 2) + int(crop_size / 2)
        y_min = int(image_size[1] / 2) - int(crop_size / 2)
        y_max = int(image_size[1] / 2) + int(crop_size / 2)
    elif location == "top left":
        x_min = 0
        x_max = int(crop_size)
        y_min = int(image_size[1]) - int(crop_size)
        y_max = image_size[1]
    elif location == "top right":
        x_min = int(image_size[0]) - int(crop_size)
        x_max = image_size[0]
        y_min = int(image_size[1]) - int(crop_size)
        y_max = image_size[1]
    elif location == "bottom left":
        x_min = 0
        x_max = int(image_size[0]) + int(crop_size)
        y_min = 0
        y_max = int(image_size[1]) + int(crop_size)
    else:
        x_min = 0
        x_max = int(image_size[0]) + int(crop_size)
        y_min = int(image_size[1]) - int(crop_size)
        y_max = image_size[1]
    return x_min, x_max, y_min, y_max

In [ ]:
file_number = 50  # arbitrary
z_plane = 10  # middle of stacks

In [ ]:
# we can now use various plotting commands to select a z-slice and plot the cell mask vs number of puncta

analysis_file_c1 = "/home/jbeckwith/Documents/Dropbox/Cambridge University Dropbox/Joseph Beckwith/Chemistry/Lee/Data/Sycamore/20240522_Isoforms_PSD95_Psyn/Data/With_Stain/PD_analysis/spot_analysis_psyn.csv"
puncta_colour1 = pl.read_csv(analysis_file_c1)

file_list_c1 = np.unique(np.sort(puncta_colour1["image_filename"].to_numpy()))
file_c1 = file_list_c1[file_number]  # we take a file we're interested in
raw_image_c1 = IO.read_tiff_tophotons(file_c1)[:, :, z_plane]
puncta_colour1 = puncta_colour1.filter(
    (puncta_colour1["image_filename"] == file_c1) & (puncta_colour1["z"] == z_plane + 1)
)

# now find the corresponding large object
# assume loMask is in same folder as analysis

mask_file_colour1 = os.path.join(
    os.path.split(analysis_file_c1)[0],
    os.path.split(file_c1.split(".tif")[0] + "_loMask.tiff")[1],
)
# read it in
lo_mask_c1 = IO.read_tiff(mask_file_colour1)[:, :, z_plane]

analysis_file_c2 = "/home/jbeckwith/Documents/Dropbox/Cambridge University Dropbox/Joseph Beckwith/Chemistry/Lee/Data/Sycamore/20240522_Isoforms_PSD95_Psyn/Data/With_Stain/PD_analysis/spot_analysis_isoform.csv"
puncta_colour2 = pl.read_csv(analysis_file_c2)

file_list_c2 = np.unique(np.sort(puncta_colour2["image_filename"].to_numpy()))
matching_string = ("_").join(file_c1.split(".tif")[0].split("_")[:-1])
file_c2 = [name for name in file_list_c2 if matching_string in name][0]
raw_image_c2 = IO.read_tiff_tophotons(file_c2)[:, :, z_plane]
puncta_colour2 = puncta_colour2.filter(
    (puncta_colour2["image_filename"] == file_c2) & (puncta_colour2["z"] == z_plane + 1)
)

mask_file_colour2 = os.path.join(
    os.path.split(analysis_file_c2)[0],
    os.path.split(file_c2.split(".tif")[0] + "_loMask.tiff")[1],
)
# read it in
lo_mask_c2 = IO.read_tiff(mask_file_colour2)[:, :, z_plane]

In [ ]:
plotter = PlottingFunctions.Plotter()

fig, axs = plotter.two_column_plot(
    ncolumns=3, nrows=1, widthratio=[1, 1, 1], heightratio=[1]
)

crop_size = 200
location = "center"

x_min, x_max, y_min, y_max = crop_coord_gen(raw_image_c1.shape, crop_size, location)

puncta_colour1_toplot = puncta_colour1.filter(
    (puncta_colour1["x"] >= x_min)
    & (puncta_colour1["x"] <= x_max)
    & (puncta_colour1["y"] <= y_max)
    & (puncta_colour1["y"] >= y_min)
)
axs[0] = plotter.image_scatter_plot(
    axs=axs[0],
    xdata=puncta_colour1_toplot["x"] - x_min,
    ydata=puncta_colour1_toplot["y"] - y_min,
    data=raw_image_c1[x_min:x_max, y_min:y_max],
    cbarlabel=r"intensity (photons)",
    s=5,
    lws=1.25,
    scattercolor="#4ee44e",
)
axs[0].set_xlim([0, crop_size - 1])
axs[0].set_ylim([0, crop_size - 1])

puncta_colour2_toplot = puncta_colour2.filter(
    (puncta_colour2["x"] >= x_min)
    & (puncta_colour2["x"] <= x_max)
    & (puncta_colour2["y"] <= y_max)
    & (puncta_colour2["y"] >= y_min)
)
axs[1] = plotter.image_scatter_plot(
    axs=axs[1],
    xdata=puncta_colour2_toplot["x"] - x_min,
    ydata=puncta_colour2_toplot["y"] - y_min,
    data=raw_image_c2[x_min:x_max, y_min:y_max],
    cbarlabel=r"intensity (photons)",
    s=5,
    lws=1.25,
    scattercolor="#4e4ee4",
)
axs[1].set_xlim([0, crop_size - 1])
axs[1].set_ylim([0, crop_size - 1])


axs[2] = plotter.image_scatter_plot(
    axs=axs[2],
    xdata=puncta_colour1_toplot["x"] - x_min,
    ydata=puncta_colour1_toplot["y"] - y_min,
    data=np.zeros_like(raw_image_c2[x_min:x_max, y_min:y_max]),
    cbarlabel=r"",
    s=5,
    lws=1.25,
    scattercolor="#4ee44e",
    cbar="off",
)

axs[2] = plotter.image_scatter_plot(
    axs=axs[2],
    xdata=puncta_colour2_toplot["x"] - x_min,
    ydata=puncta_colour2_toplot["y"] - y_min,
    data=np.zeros_like(raw_image_c2[x_min:x_max, y_min:y_max]),
    cbarlabel=r" ",
    s=5,
    lws=1.25,
    scattercolor="#4e4ee4",
    cbar="off",
)
axs[2].set_xlim([0, crop_size - 1])
axs[2].set_ylim([0, crop_size - 1])

plt.tight_layout(h_pad=-15)